# Sergio's Week 3

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Get page information

In [12]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
Page= BeautifulSoup(source, "lxml")

Get the Table information

In [13]:
Table = Page.find('table')
headings = [th.get_text().strip() for th in Table.find("tr").find_all("th")]
headings = np.array(headings)
A=headings.shape
rows = [td.get_text().strip() for td in Table.tbody.find_all("td")]
rows =np.array(rows)
B=rows.shape

Prepare the Dataframe

In [14]:
df = pd.DataFrame(np.array(rows).reshape(int(B[0]/A[0]),A[0],), columns=headings)

Ignore cells with a borough that is Not assigned.

In [15]:
df=df[df.Borough != 'Not assigned']

if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [16]:
df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough

combined into one row with the neighborhoods separated with a comma

In [17]:
df=df.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()
#df[0:3]

Print Shape

In [18]:
#print(df.shape)

Import geocoder and create new columns

In [19]:
import geocoder # import geocoder
df['Latitude'] = None
df['Longitude'] = None

Obtain the coordinates

In [20]:
Codes=df.Postcode.unique().tolist()
counter=0
while counter < 500:
    for i in range(len(Codes)):
        # initialize your variable to None
        lat_lng_coords = None
        # loop until you get the coordinates
        while (counter < 500) and (lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(Codes[i]))
            lat_lng_coords = g.latlng
            counter=counter+1
            
        if lat_lng_coords != None:
            df.Latitude[i] = lat_lng_coords[0]
            df.Longitude[i] = lat_lng_coords[1]
            
            df.Latitude[df.Postcode == Codes[i]] = lat_lng_coords[0]
            df.Longitude[df.Postcode == Codes[i]] = lat_lng_coords[1]

If iterations are to many obtain data from csv

In [21]:
if counter == 500:
    df2=pd.read_csv('https://cocl.us/Geospatial_data')
    df = df.set_index('Postcode')
    df2 = df2.set_index('Postal Code')
    df.update(df2)
    df.reset_index(inplace=True)

Results

In [22]:
df[0:3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
